<a href="https://colab.research.google.com/github/Janani-SB/GenAI_projects-/blob/main/Mini_project_4_support_analyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Prompt Engineering with LangChain and ChatGPT for real-world tasks

In this notebook you will leverage ChatGPT and LangChain to solve and do a few mini-projects based on some real-world scenarios:

- Mini-Project 1: Review Analyst
- Mini-Project 2: Research Paper Analyst
- Mini-Project 3: Social Media Marketing Analyst
- Mini-Project 4: IT Support Analyst



# **Mini-Project 4 - IT Support Analyst**

Ask ChatGPT to act as a IT support agent, process each customer IT ticket message and output the response in JSON with the following fields

```
orig_msg: The original customer message
orig_lang: Detected language of the customer message e.g. Spanish
category: 1-2 word describing the category of the problem
trans_msg: Translated customer message in English
response: Response to the customer in orig_lang
trans_response: Response to the customer in English
```

Try to use a JSON parser to get the responses in JSON for each ticket

In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.65
    Uninstalling langchain-core-0.3.65:
      Successfully uninstalled langchain-core-0.3.65
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25


In [ ]:
from getpass import getpass
OPENAI_KEY = getpass("Enter your API Key here :")

Enter your API Key here :··········


In [ ]:
import os
os.environ['OPENAI_API_KEY'] = OPENAI_KEY

# Load necessary dependencies and ChatGPT LLM

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# LLM Model

In [ ]:
chatgpt = ChatOpenAI(model_name='gpt-4', temperature=0.9)

### Access Customer IT Support ticket data

In [ ]:
it_support_queue = [
    "Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.",
    "எனக்கு வைஃபை மூலம் இணைய அணுகல் இல்லை. சிவப்பு விளக்கு ஒளிர்கிறது."
    "Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.",
    "プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。",
    "Я не могу войти в систему учета времени, появляется сообщение об ошибке. Мне нужна помощь.",
    "Internet bağlantım çok yavaş ve bazen tamamen kesiliyor. Yardım eder misiniz?",
    "Не могу установить обновление безопасности. Появляется код ошибки. Помогите, пожалуйста."
]

# Define Output Parser

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

#Base model
orig_msg: The original customer message

orig_lang: Detected language of the customer message e.g. Spanish

category: 1-2 word describing the category of the problem

trans_msg: Translated customer message in English

response: Response to the customer in orig_lang

trans_response: Response to the customer in English

In [ ]:
class ITSupportResponse(BaseModel):
    orig_msg: str = Field(description= "The Original customer IT support query message")
    orig_lang: str = Field(description="Detected language of the customer message e.g. Spanish")
    category: str = Field(description="1-2 word describing the category of the problem")
    trans_msg: str = Field(description="Translated customer IT support query message in English")
    response: str = Field(description= "Response to the customer in their original language - orig_lang")
    trans_response: str = Field(description="Response to the customer in English")

parser = JsonOutputParser(pydantic_object=ITSupportResponse)

In [ ]:
parser

JsonOutputParser(pydantic_object=<class '__main__.ITSupportResponse'>)

In [ ]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"orig_msg": {"title": "Orig Msg", "description": "The Original customer IT support query message", "type": "string"}, "orig_lang": {"title": "Orig Lang", "description": "Detected language of the customer message e.g. Spanish", "type": "string"}, "category": {"title": "Category", "description": "1-2 word describing the category of the problem", "type": "string"}, "trans_msg": {"title": "Trans Msg", "description": "Translated customer IT support query message in English", "type": "string"}, "response": {"title": "Response

# Create the input prompt for the LLM

In [ ]:
prompt_txt = """
Act as an Information Technology (IT) customer support agent.
For the IT support message mentioned below
use the following output format when generating the output response

Output Format Instructions:
{format_instructions}

Customer IT support message:
{it_support_msg}
"""

In [ ]:
prompt = PromptTemplate(template = prompt_txt,
                        input_variables=["it_support_msg"],
                        partial_variables= {"format_instructions": parser.get_format_instructions()},)

# Create a LCEL LLM Chain

In [ ]:
llm_chain = (prompt | chatgpt | parser)

In [ ]:
formated_msg = [{"it_support_msg": msg} for msg in it_support_queue]
formated_msg[0]

{'it_support_msg': 'Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.'}

In [ ]:
formated_msg[2]

{'it_support_msg': 'プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。'}

In [ ]:
responses = llm_chain.map().invoke(formated_msg)

In [ ]:
print(responses[0])

{'orig_msg': 'Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.', 'orig_lang': 'Portuguese', 'category': 'Sync Issue', 'trans_msg': 'I can not sync my contacts with the phone. I always receive a failure message.', 'response': 'Sinto muito por você estar enfrentando esse problema. Tente reiniciar o telefone e verificar se a sincronização está ativada nas configurações. Se o problema persistir, entre em contato novamente.', 'trans_response': "I'm sorry you're experiencing this issue. Try restarting your phone and check if sync is enabled in the settings. If the problem persists, please contact again."}


In [ ]:
responses[1]

{'orig_msg': 'எனக்கு வைஃபை மூலம் இணைய அணுகல் இல்லை. சிவப்பு விளக்கு ஒளிர்கிறது.Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.',
 'orig_lang': 'Tamil, Italian',
 'category': 'Wifi, Printer',
 'trans_msg': "I do not have internet access via wifi. The red light is on. I'm having trouble printing documents remotely. The job is not sent to the network printer.",
 'response': 'உங்கள் வைஃபை மேலாண்மையரை அழைத்து வருவதை செய்யவும், அவர்கள் குறிப்பிட்ட அமைப்பை மீண்டும் அமைக்க உதவுவார்கள். பின்னர், உங்கள் பிரிண்டரை உங்கள் கணினியில் மீண்டும் அமைக்கவும். நேரடியாக கணினிக்கு முதலில் பிரிண்ட் செய்வதையும், நீட்வொர்க் மூலம் பிரிண்ட் செய்வதையும் சோதிக்கவும். Ho problemi con la stampante, prova a riavviare la stampante e il computer. Inoltre, ti consigliamo di controllare il collegamento alla rete e le impostazioni della stampante.',
 'trans_response': "Please call your wifi manager, they will help you to reset the specified settings. Then, reconfigure your

In [ ]:
responses[2]

{'orig_msg': 'プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。',
 'orig_lang': 'Japanese',
 'category': 'Printer Problem',
 'trans_msg': 'I have changed the toner of the printer, but the print quality has decreased. I need support.',
 'response': '理解しました。印刷品質が低下する原因はいくつかあります。トナーの種類を確認し、プリンターの設定をリセットしてみてください。それでも問題が解決しない場合は、詳細を教えていただけますか？',
 'trans_response': 'Understood. There are several reasons why print quality could decrease. Please check the type of toner and try resetting your printer settings. If the problem still persists, can you provide more details?'}

In [ ]:
import pandas as pd
df = pd.DataFrame(responses)
df

,orig_msg,orig_lang,category,trans_msg,response,trans_response
0,Não consigo sincronizar meus contatos com o te...,Portuguese,Sync Issue,I can not sync my contacts with the phone. I a...,Sinto muito por você estar enfrentando esse pr...,I'm sorry you're experiencing this issue. Try ...
1,எனக்கு வைஃபை மூலம் இணைய அணுகல் இல்லை. சிவப்பு ...,"Tamil, Italian","Wifi, Printer",I do not have internet access via wifi. The re...,உங்கள் வைஃபை மேலாண்மையரை அழைத்து வருவதை செய்யவ...,"Please call your wifi manager, they will help ..."
2,プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。,Japanese,Printer Problem,"I have changed the toner of the printer, but t...",理解しました。印刷品質が低下する原因はいくつかあります。トナーの種類を確認し、プリンターの設...,Understood. There are several reasons why prin...
3,"Я не могу войти в систему учета времени, появл...",Russian,Login Issue,"I can't log into the time tracking system, I'm...","Здравствуйте, простите за неудобства. Вы может...","Hello, sorry for the inconvenience. Can you sp..."
4,Internet bağlantım çok yavaş ve bazen tamamen ...,Turkish,Internet Issue,My internet connection is very slow and someti...,"Evet, yardımcı olabilirim. İlk önce, modeminiz...","Yes, I can help. First off, have you tried reb..."
5,Не могу установить обновление безопасности. По...,Russian,Update Installation,I can't install the security update. An error ...,Попробуйте перезагрузить устройство и повторно...,Try restarting your device and reinstalling th...


In [ ]:
df.to_csv("IT_Support_Query_Response.csv")